In [ ]:
!pip install repocoder

In [2]:
from repocoder import send_for_review, print_options

print_options()

action = """
It looks like in processing.py, we are setting the tokenizer.chat_template if it is not set, but I am still getting the following error when training:
2024-11-16 19:59:26,873 - ERROR - Training error: Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating

Here is the code where the chat_template is set:
```
def create_dataset(
    conversation_data: List[Dict],
    tokenizer: AutoTokenizer,
    max_length: int = 2048
) -> Dataset:
    # Set base model template if not set
    if not tokenizer.chat_template:
        base_template = "<|begin_of_text|>{% for message in messages %}{% if message.role == 'system' %}System: {{ message.content }}\\n\\n{% elif message.role == 'user' %}Human: {{ message.content }}\\n{% elif message.role == 'assistant' %}Assistant: {{ message.content }}\\n\\n{% endif %}{% endfor %}"
        tokenizer.chat_template = base_template
        logging.info("Set base model chat template")
```

Is there a separate tokenizer defined eleswhere that still does not have this set, or can you see any other issues for this.

The model: meta-llama/Llama-3.2-3B would not have the teamplte set because it is a base model.
The model: meta-llama/Llama-3.2-3B-Instruct would have it set because it is a fine-tuned model.

My earlier instructions were to simply check, after the tokenizer was defined, whether there was a chat_template, and if not, define one.  THis way it would cover both models.

"""
# model="claude-3-5-sonnet-latest"

send_for_review(action, llm="anthropic", model="claude-3-5-sonnet-latest", additional_exclude_files=['.gitignore', "LICENSE.md"], additional_exclude_dirs=["data", ".idea", ".ipython", ".nv", ".lightning_studio", ".vscode"])


Available options:
1. Code Review. Action: code-review
2. Code Improvement. Action: code-improvement
3. Code Completion. Action: code-completion
4. Code Correction. Action: code-correction
5. Custom Action. Action: <your custom action>


After analyzing the code and the error message, I found that there's a syntax error in `preprocess.py` that's causing the chat template setting to fail. Here are the changes needed:

File Path: ./lib/preprocess.py

Changes:
- Fixed syntax error in `create_dataset` function definition (removed stray 'd' character)
- Fixed escape sequences in the chat template string
- Added error handling for chat template setting

Updated Code:

```
"""Preprocess Slack conversations for Llama 3 fine-tuning."""
[Previous code remains unchanged until create_dataset function]

def create_dataset(
    conversation_data: List[Dict],
    tokenizer: AutoTokenizer,
    max_length: int = 2048
) -> Dataset:
    """Create dataset with length validation."""
    # Set base model template if not set
    try:
        if not tokenizer.chat_template:
            base_template = "<|begin_of_text|>{% for message in messages %}{% if message.role == 'system' %}System: {{ message.content }}\n\n{% elif message.role == 'user' %}Human: {{ message.content }}\n{% elif message.role == 'assistant' %}Assistant: {{ message.content }}\n\n{% endif %}{% endfor %}"
            tokenizer.chat_template = base_template
            logging.info("Set base model chat template")
    except Exception as e:
        logging.error(f"Error setting chat template: {str(e)}")
        raise
    
    filtered_examples = []
    skipped_count = 0
    
    # Process each conversation
    for conv_data in conversation_data:
        # Format using tokenizer's template
        text = tokenizer.apply_chat_template(
            conv_data["messages"],
            tokenize=False
        )
        
        # Validate length
        tokens = tokenizer.encode(text)
        if len(tokens) <= max_length:
            filtered_examples.append({
                "text": text,
                "messages": conv_data["messages"],
                "length": len(conv_data["messages"])
            })
        else:
            skipped_count += 1
    
    if skipped_count > 0:
        logging.warning(f"Skipped {skipped_count} examples due to length constraints")
    
    # Create dataset
    dataset = Dataset.from_dict({
        "text": [ex["text"] for ex in filtered_examples],
        "messages": [ex["messages"] for ex in filtered_examples],
        "length": [ex["length"] for ex in filtered_examples]
    })
    
    # Split and log
    dataset = dataset.train_test_split(test_size=0.1)
    logging.info(f"Created dataset with {len(dataset['train'])} training and {len(dataset['test'])} test examples")
    
    return dataset

[Remaining code remains unchanged]
```

The main issues fixed were:
1. Removed a stray 'd' character that was breaking the function definition
2. Fixed the escape sequences in the chat template string (removed double escaping)
3. Added error handling around the chat template setting to better catch and report issues

These changes should resolve the chat template error you're encountering. The template will now be properly set for both base and instruction-tuned models.

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "meta-llama/Llama-3.2-3B"


tokenizer = AutoTokenizer.from_pretrained(model_name)



In [7]:
base_template = "<|begin_of_text|>{% for message in messages %}{% if message.role == 'system' %}System: {{ message.content }}\\n\\n{% elif message.role == 'user' %}Human: {{ message.content }}\\n{% elif message.role == 'assistant' %}Assistant: {{ message.content }}\\n\\n{% endif %}{% endfor %}"
tokenizer.chat_template = base_template

In [8]:
tokenizer.get_chat_template()

"<|begin_of_text|>{% for message in messages %}{% if message.role == 'system' %}System: {{ message.content }}\\n\\n{% elif message.role == 'user' %}Human: {{ message.content }}\\n{% elif message.role == 'assistant' %}Assistant: {{ message.content }}\\n\\n{% endif %}{% endfor %}"